In [1]:
import types
import numpy as np
import nltk
import re
import random
import pandas as pd
# from gensim.parsing.porter import PorterStemmer
from nltk.corpus import gutenberg, stopwords
from nltk.probability import FreqDist
from nltk.corpus import words
import matplotlib.pyplot as plt
import requests

# from gensim.models import Word2Vec
from sklearn import tree
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import KNeighborsClassifier

# !python -m spacy download en_core_web_sm
# import en_core_web_sm


c:\Users\river\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\river\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\river\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\river\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [13]:
import string
word = "hospital"
lower_upper_alphabet = string.ascii_letters

for _ in range(1):
  random_letter = random.choice(lower_upper_alphabet)
  i = random.randint(1,len(word))
  word = word[:i] + random_letter.lower() + word[i:]
books=[]
urls=[]

urls.append("https://www.gutenberg.org/files/51660/51660-0.txt")
# urls.append("https://www.gutenberg.org/files/20023/20023-8.txt")
# urls.append("https://www.gutenberg.org/files/36641/36641-8.txt")
# urls.append("https://www.gutenberg.org/files/63444/63444-0.txt")
# urls.append("https://www.gutenberg.org/files/1265/1265.txt")

for url in urls:
  books.append(requests.get(url).text)

In [14]:
import string

def apply_error(word, remove=0.3, add=0.3, swap_letter=True):
  def swap(s, i, j):
    return ''.join((s[:i], s[j], s[i+1:j], s[i], s[j+1:]))

  if swap_letter:
    i = random.randint(0,len(word)-1)
    j = random.randint(0,len(word)-1)
    word = swap(word, i, j)

  letters_to_remove = int(remove*len(word))
  letters_to_add = int(add*len(word))

  for _ in range(letters_to_remove):
    i = random.randint(1,len(word))
    word = word[:i-1] + word[i:]

  lower_upper_alphabet = string.ascii_letters
  for _ in range(letters_to_add):
    random_letter = random.choice(lower_upper_alphabet)
    i = random.randint(1,len(word))
    word = word[:i] + random_letter.lower() + word[i:]

  return word

def prepare_data(text):
  # to lower case
  text = text.lower()
  
  # Remove non-ascii characters
  text = text.encode("ascii", "ignore").decode()

  # Remove \n and \r
  text = re.sub(r'[\r*\n*]+', ' ', text)

  # Remove websites and hashtags
  text = re.sub(r'_|#|http\S+|www\S+', ' ', text)

  # Substitute numbers by keyword num
  text = re.sub(r'\d+', 'num', text)

  
  # Divide into sentences
  # sents = text_to_sent_list(text, nlp)

  # Apply tokenization
  regex = r"\w+'?\w+\s"
  words = re.findall(regex,text)
  words = [w[:-1] for w in words]
  words = [w for w in words if len(w)>1 or w in ["a", "i"]]

  unique = np.unique(words, return_counts=True)
  [[w, c] for w, c in zip(*unique) if  len(w) == 2]

  return words

words = [prepare_data(book) for book in books]
words = [y for x in words for y in x]


words = set(words)

# multiplication_factor = 3
# for i in range(3):
#   words


In [15]:
correct_words = list(set(prepare_data(books[0])))
my_wrong1 =[]
my_wrong2 =[]
my_wrong3 =[]
my_wrong4 =[]
my_wrong5 =[]
for correct in correct_words:
  
  my_wrong1.append(apply_error(correct, remove=0.1, add=0.1,swap_letter=True))
  my_wrong2.append(apply_error(correct, remove=0.1, add=0.1, swap_letter=True))
  my_wrong3.append(apply_error(correct, remove=0.2, add=0.1))
  my_wrong4.append(apply_error(correct, remove=0.2, add=0.1,swap_letter=True))
  my_wrong5.append(apply_error(correct, remove=0.1, add=0.2,swap_letter=True))
  # print(correct,my_wrong)
  # wrong_data.append(my_wrong)

In [16]:
df = pd.DataFrame({"correct":correct_words,"wrong1":my_wrong1,"wrong2":my_wrong2,"wrong3":my_wrong3,"wrong4":my_wrong4,"wrong5":my_wrong5})
df

,correct,wrong1,wrong2,wrong3,wrong4,wrong5
0,unimagined,unuinimzained,unimamgagineb,nnimainedn,unimainleed,gnimmauiegd
1,forests,faorestests,foserts,orrests,sorefs,forestprsetso
2,fainted,ftinaed,faintientdd,naifed,fniate,faintder
3,numerous,numersuo,numerwoeuous,nuhmerrou,ntmeruomrous,umieerrousr
4,tallest,tllaelsest,tallllest,telast,alelesttalest,tatlejsl
...,...,...,...,...,...,...
10976,therefor,ehertfor,thfreeor,tthrefor,terefor,teerihfor
10977,doves,dooves,dovess,ovvees,dvse,sovedr
10978,bedecked,beddckee,beecekczked,beedeked,edeckbdeckezd,bedreckedd
10979,revelry,rreevelry,remvelreyelry,revely,reveorleery,revfeellry


In [17]:
def myWord2vec(word):
    vector  = [0]*26    
    for c in word:
        index = ord(c)-97
        if index>=26 or index <0:
            continue
        vector[index]+=1
    return vector


In [18]:
myWord2vec("your're")

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0]

In [4]:
myWord2vec("aaahmed")

[3, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [5]:
myWord2vec("ahamed")

[2, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [6]:
import numpy as np
from numpy.linalg import norm
 
# define two lists or array
A = np.array(myWord2vec("sendom"))
B = np.array(myWord2vec("sand"))
print("A:\n", A)
print("B:\n", B)
 
# compute cosine similarity
cosine = np.dot(A,B)/(norm(A, axis=0)*norm(B))
print("Cosine Similarity:\n", cosine)

dist = np.linalg.norm(A - B)
print(dist)

A:
 [0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0]
B:
 [1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]
Cosine Similarity:
 0.6123724356957946
2.0


In [7]:
def cosin_similarity(word1,word2):
    # print(word1,word2)
    A = np.array(myWord2vec(word1))
    B = np.array(myWord2vec(word2))

    # compute cosine similarity
    cosine = np.dot(A,B)/(norm(A, axis=0)*norm(B))
    return cosine

In [8]:
def get_most_similar(word, posibilites):
    matched_word  = ""
    max_s = 0
    for w in posibilites:
        cos = cosin_similarity(word,w)
        if cos>max_s:
            max_s = cos
            matched_word = w
    return matched_word,max_s


In [24]:
from numpy.ma.core import empty
max = len(df)
print(max)
correct_words =correct_words[:max] 
wrong1_performance =[]
num_correct = 0
for i in range(max):
  correct_w = df.iloc[i,0]
  wrong = df.iloc[i,1]
  flag = 0
  close_matches, cos_value = get_most_similar(wrong, correct_words)
  
  # print(close_matches, cos_value)
  if close_matches is not empty and close_matches == correct_w:
    num_correct+=1
    flag = 1
  wrong1_performance.append((flag, close_matches, cos_value))
    

10981


In [25]:
num_correct

9280

In [26]:
len(wrong1_performance)

10981

In [27]:
df['wrong1_performance'] = wrong1_performance
df = df[['correct', 'wrong1', 'wrong1_performance', "wrong2",'wrong3',"wrong4","wrong5"]]

In [28]:
print(num_correct)
print("accuracy: ",num_correct/max)
df

9280
accuracy:  0.8450960750387032


,correct,wrong1,wrong1_performance,wrong2,wrong3,wrong4,wrong5
0,unimagined,unuinimzained,"(1, unimagined, 0.9258200997725514)",unimamgagineb,nnimainedn,unimainleed,gnimmauiegd
1,forests,faorestests,"(1, forests, 0.9456108576893003)",foserts,orrests,sorefs,forestprsetso
2,fainted,ftinaed,"(1, fainted, 0.9999999999999999)",faintientdd,naifed,fniate,faintder
3,numerous,numersuo,"(1, numerous, 0.9999999999999998)",numerwoeuous,nuhmerrou,ntmeruomrous,umieerrousr
4,tallest,tllaelsest,"(0, tells, 0.9669875568304563)",tallllest,telast,alelesttalest,tatlejsl
...,...,...,...,...,...,...,...
10976,therefor,ehertfor,"(1, therefor, 1.0000000000000002)",thfreeor,tthrefor,terefor,teerihfor
10977,doves,dooves,"(1, doves, 0.9486832980505138)",dovess,ovvees,dvse,sovedr
10978,bedecked,beddckee,"(1, bedecked, 1.0)",beecekczked,beedeked,edeckbdeckezd,bedreckedd
10979,revelry,rreevelry,"(1, revelry, 0.9869275424396536)",remvelreyelry,revely,reveorleery,revfeellry


In [29]:
from scipy.spatial import distance
a = (1, 2, 3)
b = (4, 5, 6)
dst = distance.euclidean(a, b)

In [30]:
distance.euclidean(myWord2vec("ahmed"),myWord2vec("hamed"))

0.0

In [31]:
cosin_similarity("ahmed","hamed")

0.9999999999999998

In [32]:
cosin_similarity("revolst","revolts")


0.9999999999999999

In [33]:
get_most_similar("abunddanjc",correct_words)

('abundance', 0.9013878188659974)